In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from config import Config
from peft import  LoraConfig, get_peft_model
from data_module import DualBatchDataset
from collators import custom_data_collator_interleaved
from utils import find_all_linear_names
from forget_trainer import BatchGradDiffTrainer
from accelerate import Accelerator
import pandas as pd


In [3]:
cfg = Config()

#accelerator = Accelerator()

In [4]:
forget = pd.read_csv('/home/praveen/theoden/emnlp25/forget_20_1.csv')
retain = pd.read_csv(cfg.retain_path)
test = pd.read_csv(cfg.test_path)

In [34]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct', token = cfg.access_token)
tokenizer.pad_token = tokenizer.eos_token

In [35]:
question = forget['question'][0]
answer = forget['answer'][0]
messages = [{"role": "user", "content": question}]


new_question_tokenize = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generataion_prompt=True
    )

new_question_tokenizer = tokenizer.apply_chat_template(
        messages,
        tokenizer = False,
        add_generataion_prompt=True
    )


print(new_question_tokenize)
print(new_question_tokenizer)



<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What nationality was Benedetto Varchi?<|eot_id|>
[128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 3923, 59343, 574, 52720, 53979, 650, 1132, 72, 30, 128009]


In [36]:
full_text_tokenizer = str(new_question_tokenizer) + answer
full_text_tokenize = str(new_question_tokenize) + answer

print(full_text_tokenizer)
print(full_text_tokenize)

[128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 3923, 59343, 574, 52720, 53979, 650, 1132, 72, 30, 128009]Italian
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What nationality was Benedetto Varchi?<|eot_id|>Italian


In [38]:
num_question_tokens = len(tokenizer.tokenize(str(new_question_tokenize), add_special_tokens=True))
num_question_tokens

41

In [39]:
encoded_tokenizer  = tokenizer(
        full_text_tokenizer,
        add_special_tokens=True,
        max_length=256,
        truncation=True,
    )

encoded_tokenize = tokenizer(
        full_text_tokenize,
        add_special_tokens=True,
        max_length=256,
        truncation=True,
)

In [40]:
encoded_tokenizer

{'input_ids': [128000, 58, 4386, 931, 11, 220, 4386, 11030, 11, 220, 22750, 20, 11, 220, 4386, 11194, 11, 220, 15828, 11, 220, 20062, 2287, 11, 220, 5894, 18, 11, 220, 10568, 914, 11, 220, 16955, 21, 11, 220, 914, 11, 220, 25136, 15, 11, 220, 8610, 11, 220, 14087, 21, 11, 220, 972, 11, 220, 3753, 11, 220, 10895, 1187, 11, 220, 16955, 21, 11, 220, 914, 11, 220, 8610, 11, 220, 10674, 22, 11, 220, 4278, 5495, 11, 220, 8610, 11, 220, 14087, 21, 11, 220, 777, 11, 220, 15828, 11, 220, 4386, 13858, 11, 220, 4386, 11030, 11, 220, 23213, 11, 220, 4386, 11194, 11, 220, 15828, 11, 220, 19695, 18, 11, 220, 22608, 3391, 11, 220, 23402, 11, 220, 22369, 508, 11, 220, 23033, 4643, 11, 220, 13655, 11, 220, 8190, 17, 11, 220, 5332, 11, 220, 966, 11, 220, 4386, 13858, 60, 70211], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [41]:
encoded_tokenize

{'input_ids': [128000, 128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 3923, 59343, 574, 52720, 53979, 650, 1132, 72, 30, 128009, 70211], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [42]:
decoded_tokenize = tokenizer.decode(encoded_tokenize['input_ids'], skip_special_tokens=False)
decoded_tokenizer = tokenizer.decode(encoded_tokenizer['input_ids'], skip_special_tokens=False)


print(decoded_tokenize)
print(decoded_tokenizer)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What nationality was Benedetto Varchi?<|eot_id|>Italian
<|begin_of_text|>[128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 3923, 59343, 574, 52720, 53979, 650, 1132, 72, 30, 128009]Italian


In [44]:
blah = len(tokenizer.tokenize(str(new_question_tokenize), add_special_tokens=False))
print(blah)

40


In [45]:
bl = tokenizer.tokenize(str(new_question_tokenize), add_special_tokens=False)
print(bl)

['<|begin_of_text|>', '<|start_header_id|>', 'system', '<|end_header_id|>', 'ĊĊ', 'Cut', 'ting', 'ĠKnowledge', 'ĠDate', ':', 'ĠDecember', 'Ġ', '202', '3', 'Ċ', 'Today', 'ĠDate', ':', 'Ġ', '26', 'ĠJul', 'Ġ', '202', '4', 'ĊĊ', '<|eot_id|>', '<|start_header_id|>', 'user', '<|end_header_id|>', 'ĊĊ', 'What', 'Ġnationality', 'Ġwas', 'ĠBened', 'etto', 'ĠV', 'arch', 'i', '?', '<|eot_id|>']


In [ ]:
forget['tokenized'] = ''

for i, row in forget.iterrows():
    question = row['question']
    answer = row['answer']
    messages = [{"role": "user", "content": question}]
    new_question = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generataion_prompt=True
    )
    full_text = str(new_question) + answer
    
    full_text_len = len(tokenizer.tokenize(full_text, add_special_tokens=False))
    print
    
    forget.loc[i, 'tokenized'] = new_question

In [ ]:
messages = [{"role": "user", "content": question}]
    new_question = tokenizer.apply_chat_template(
        messages,
        tokenizer = False,
        add_generataion_prompt=True
    )

In [4]:
cfg.save_dir = 'outputs/wpu_ails_grad_diff_model'

In [4]:
print('loading the paths to forget, retain and test set')
forget = pd.read_csv(cfg.forget_path) #cfg.forget_path
retain = pd.read_csv(cfg.retain_path) #cfg.retain_path

loading the paths to forget, retain and test set


In [6]:
print(f"\nLoading the Tokenizer {cfg.model_id}")
tokenizer = AutoTokenizer.from_pretrained(cfg.model_id, token = cfg.access_token)
tokenizer.pad_token = tokenizer.eos_token


Loading the Tokenizer praveensonu/llama_3_1_8b_finetuned


In [7]:
print(f"\nLoading the Model {cfg.model_id}")
model = AutoModelForCausalLM.from_pretrained(cfg.model_id, 
                                             torch_dtype = torch.bfloat16, 
                                             token=cfg.access_token,)


Loading the Model praveensonu/llama_3_1_8b_finetuned


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
config = LoraConfig(
        r = cfg.LoRA_r,
        lora_alpha = cfg.LoRA_alpha,
        lora_dropout= cfg.LoRA_dropout,
        target_modules = find_all_linear_names(model),
        bias = 'none',
        task_type = 'CAUSAL_LM',
    )


In [9]:
model = get_peft_model(model, config)
model.print_trainable_parameters()
#model.generation_config.do_sample = True
model.config.use_cache = False

trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605


In [10]:
forget['factor'] = -1.0
retain['factor'] = 1.0
forget['factor'] = forget['factor'].astype('float')
retain['factor'] = retain['factor'].astype('float')
retain['idk'] = 'idk'

In [11]:
total_batch_size = 8
n_forget_in_batch = 1
n_retain_in_batch = total_batch_size - n_forget_in_batch
print(f"Batch size: {total_batch_size}, Forget samples in batch: {n_forget_in_batch}, Retain samples in batch: {n_retain_in_batch}")

Batch size: 8, Forget samples in batch: 1, Retain samples in batch: 7


In [12]:
train_dataset =  DualBatchDataset(
    forget_df = forget,
    retain_df = retain,
    tokenizer = tokenizer,
    max_length = 256,
    block_size = total_batch_size,
    n_forget   = n_forget_in_batch,
    n_retain   = n_retain_in_batch
)

Combined dataset initialized with 2112 samples.
Verifying sample structure (first few blocks):
  Block 0: 1 forget, 7 retain samples. Expected: 1, 7
  Block 1: 1 forget, 7 retain samples. Expected: 1, 7
  Block 2: 1 forget, 7 retain samples. Expected: 1, 7


In [13]:
training_args = TrainingArguments(
        output_dir = cfg.save_dir,
        overwrite_output_dir= True,
        learning_rate = cfg.lr,
        per_device_train_batch_size= 8, 
        num_train_epochs= 10,
        weight_decay = cfg.weight_decay,
        logging_dir = f'{cfg.save_dir}/logs',
        eval_strategy= 'no',
        label_names = ['labels'],
        bf16 = True,
        gradient_accumulation_steps= 1,
        #save_only_model=True,
        report_to = 'wandb',
    )

In [14]:
trainer = BatchGradDiffTrainer(
        model = model,
        args = training_args,
        train_dataset = train_dataset,
        tokenizer = tokenizer,
        data_collator = custom_data_collator_interleaved,
    )


/home/praveen/theoden/emnlp_25/forget_trainer.py:93: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `BatchGradDiffTrainer.__init__`. Use `processing_class` instead.
  super().__init__(model= model, **hf_trainer_kwargs)


[2025-05-18 14:07:37,458] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: /usr/local/cuda-12/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: /usr/local/cuda-12/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXX

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Rank 0: Instantiating SequentialSampler for single GPU.


wandb: Currently logged in as: praveenbushipaka942 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,2.244300
1000,0.587000
1500,-1.870900
2000,-3.957400
2500,-5.431600


TrainOutput(global_step=2640, training_loss=-1.8953087546608665, metrics={'train_runtime': 2484.3452, 'train_samples_per_second': 8.501, 'train_steps_per_second': 1.063, 'total_flos': 2.4414248075526144e+17, 'train_loss': -1.8953087546608665, 'epoch': 10.0})

In [16]:
print(f'\nForget LoRA adapter saved at {cfg.save_dir}')
model.save_pretrained(cfg.save_dir)
tokenizer.save_pretrained(cfg.save_dir)


Forget LoRA adapter saved at outputs/wpu_ails_grad_diff_model


('outputs/wpu_ails_grad_diff_model/tokenizer_config.json',
 'outputs/wpu_ails_grad_diff_model/special_tokens_map.json',
 'outputs/wpu_ails_grad_diff_model/tokenizer.json')